<h2 align="center">DLOps - Exportr um modelo a ONNX</h2>

Data Scientist.: PhD.Eddy Giusepe Chirinos Isidro

# ONNX

Depois de treinar vários modelos, compará-los e decidir qual usaremos em produção, temos que exportá-lo. Para isso, existem alternativas, em função da aplicação (desde desplegar um modelo em dispositivos celulares ou IoT até em servidores na nuvem accesíveis através de uma API). Aqui consideramos que nosso modelo será executado num servidor em nuvem, o qual é o mais comum já que desta maneira podemos controlar os recursos computacionais disponíveis para sua execução, monitorar, desplegar novas versões facilmente, etc. Em nosso caso como treinamos os modelos usando Pytorch e Pytorch Lightning, poderíamos usar qualquer framework em Python que nos permita server as predições através de internet, como por exemplo: [Flask](https://flask.palletsprojects.com/en/2.0.x/) ou [FastAPI](https://fastapi.tiangolo.com/). O principal problema desta opção é que teremos que carregar todas as bibliotecas (e suas dependências) em nossa API, o qual resultará em uma carga muito pesada. Recentemente, Pytorch inclui uma solução dedicada para este caso de uso, [Torchserve](https://pytorch.org/serve/) que se bem nos oferece uma solução otimizada para server modelos em produção, está limitada ao uso de modelos Pytorch.


É neste ponto em que entra [ONNX](https://onnx.ai/), um standard aberto para a representação de Redes Neurais que permite a interoperabilidade  entre bibliotecas e oferece uma solução otimizada para server modelos em produção (tanto na nuvem como em dispositivos celulares). Desta maneira podemos desacoplar o treinamento de modelos de sua posta em produção, utilizando em cada caso as ferramentas preferidas para seu desenvolvimento. 

# Exportar um modelo a ONNX

Vamos a ver como podemos exportar um modelo treinado a ONNX. Em primeiro lugar, carregamos o `checkpoint` desejado (o qual foi gerado no script anterior).

In [6]:
from src import *

module = MNISTModule.load_from_checkpoint('/home/eddygiusepe/17_Pytorch/DLOps_MLOps_para_Deep_Learning/final.ckpt')
module.mlp

Sequential(
  (0): Linear(in_features=784, out_features=100, bias=True)
  (1): ReLU()
  (2): Linear(in_features=100, out_features=1, bias=True)
)

É Uma boa prática evaliar nosso modelo antes e depois de exportá-lo para ter certeza de que todo funciona corretamente.

In [7]:
import torch 

dm = MNISTDataModule(**module.hparams['datamodule'])
dm.setup()

def torch_eval():
    module.eval()
    with torch.no_grad():
        preds, labels = torch.tensor([]), torch.tensor([])
        for imgs, _labels in dm.val_dataloader():
            outputs = module.predict(imgs) > 0.5
            preds = torch.cat([preds, outputs.cpu().long()])
            labels = torch.cat([labels, _labels])

    acc = (preds == labels).float().mean()
    return acc.item()

torch_eval()

0.8999999761581421

Pytorch Lightning nos permite exportar um modelo a `ONNX` de maneira muito simples com a seguinte linha:

In [13]:
input_sample = torch.randint(0, 255, (1, 28, 28), dtype=torch.uint8)
module.to_onnx(
    'models/binary_classifier_3.onnx', # Nome do modelo a salvar
    input_sample, # exemplo de entrada
    export_params=True, # Exportar os parâmetros do modelo
    opset_version=11, # En função das ops no modelo, se pode trocar o opset
    input_names = ['input'], # Nome da entrada	para usar em produção
    output_names = ['output'],  # Nome da saída para usar em produção
    dynamic_axes={  # Para poder ter diferentes batch sizes
        'input' : {0 : 'batch_size'},
        'output' : {0 : 'batch_size'},
    },
)

Como observas temos que indicar o nome do modelo e a pasta onde queremos salvar, dar umas entradas de exemplo (que ONNX usará para identificar todas as operações que se realizará dentro do modelo e salvá-las), se queremos exportar os parâmetros do modelo, a versão do `opset` (este é o conjunto de operações suportadas, que vâ mudando a medida que se adicionam novas funcionalidades) e, opcionalmente, nomes para as entradas e saídas (isto é importante se nosso modelo tem várias entradas e/ou saídas) assim como indicar que eixos dinâmicos (útil para poder usar o modelo em produção com diferentes *bacth sizes*).

# ONNXRuntime

Una vez hemos exportado nuestro modelo podemos cargarlo y ejecutarlo usando el *ONNXRuntime*. Esta es una de las ventajas de ONNX, y es que existen *runtimes* para multiples entornos y lenguajes. Así pues, podrás entrenar el modelo en `Python` y luego desplegarlo tanto en `Python` como en la web con `Javascript`, en dipositivos moviles con `Android` o `iOS`, etc.

> En Python, puedes instalarlo con el comando `pip install onnxruntime`.

Para cargar el modelo instanciamos una `InferenceSession` con el `path` al modelo exportado. Luego, definiermos las entradas al modelo usando el un `dict` con el nombre definido en la fase de exportación. Date cuenta que ahora el modelo usará como entradas array de `NumPy`, ya que en este entorno `Pytorch` ya no existe. Si que es importante, sin embargo, que uses el mismo tamaño y tipo de datos que usaste en el entrenamiento. Por último, podemos obtener las salidas del modelo usando el método `run()`.

In [17]:
import onnxruntime as ort 
import numpy as np

# Carregar nosso Modelo
ort_session = ort.InferenceSession('./models/binary_classifier_3.onnx')

ort_inputs = {
    "input": np.random.randint(0, 255, (10, 28, 28), dtype=np.uint8),
}


# Para executá-lo
ort_output = ort_session.run(['output'], ort_inputs)
ort_output[0].shape


(10,)

E, como comentava antes, é importante evaliar o modelo para ter certeza que foi exportado bem. 

In [18]:
def onnx_eval():
    with torch.no_grad():
        preds, labels = [], torch.tensor([])
        for imgs, _labels in dm.val_dataloader():
            ort_inputs = {
                "input": imgs.numpy(),
            }
            ort_output = ort_session.run(['output'], ort_inputs)[0]
            outputs = ort_output > 0.5
            preds += outputs.astype(int).tolist()
            labels = torch.cat([labels, _labels])
    acc = (np.array(preds) == labels.numpy()).astype(float).mean()
    return acc 

onnx_eval()

0.9

En este caso obtenemos la misma métrica de evaluación que la obtenida con el checkpoint, así que podemos estar tranquilos de que el modelo se comportará bien en producción.

# Versionando Modelos

Una vez hemos exportado nuestro modelo y hemos verificado que funciona bien podemos versionarlo de la misma manera que hicimos con nuestro dataset.

Para ello, primero añadimos la carpeta `models` a nuestro repositorio.

```
dvc add models
```

De momento solo tenemos un modelo, el que acabamos de exportar, y podemos sincornizarlo con el repositorio remoto (en el que ya viven varias versiones de nuestro dataset) de la siguiente manera

```
dvc push
```

De esta manera, cualquier persona con accesso al proyecto en el que estamos trabajando podrá recuperar este modelo con el comando

```
dvc pull models.dvc
```

Puedes porbar que funcione eliminando la carpeta `models` y recuperándola con el comando anterior. Recuerda generar un nuevo tag y sincronizar con el repositorio de git.

```
git add .
git commit -m "primer modelo"
git push
git tag -a v3 -m "version 3"
git push origin --tags
```

A partir de ahora, al entrenar nuevos modelos, podemos añadrilos al repositorio con nuevo tag. Usar un modelo diferente en producción será tan sencillo como cambiar al tag adecuado, lo cual veremos más adelante.